In [1]:
import tensorflow_model_optimization as tfmot
import os
import tempfile
import tensorflow as tf
from keras.layers import Input, Dense, BatchNormalization, Reshape, Conv2D, add, LeakyReLU
from keras.models import Model, model_from_json
from keras.callbacks import TensorBoard, Callback
import scipy.io as sio 
import numpy as np
import math
import time
tf.compat.v1.reset_default_graph()

In [2]:
envir = 'outdoor' #'indoor' or 'outdoor'
# image params
img_height = 32
img_width = 32
img_channels = 2 
img_total = img_height*img_width*img_channels
# network params
residual_num = 2
encoded_dim = 32 #compress rate=1/4->dim.=512, compress rate=1/16->dim.=128, compress rate=1/32->dim.=64, compress rate=1/64->dim.=32

In [3]:
# Data loading
if envir == 'indoor':
    mat = sio.loadmat('data/DATA_Htrainin.mat') 
    x_train = mat['HT'] # array
    mat = sio.loadmat('data/DATA_Hvalin.mat')
    x_val = mat['HT'] # array
    mat = sio.loadmat('data/DATA_Htestin.mat')
    x_test = mat['HT'] # array

elif envir == 'outdoor':
    mat = sio.loadmat('data/DATA_Htrainout.mat') 
    x_train = mat['HT'] # array
    mat = sio.loadmat('data/DATA_Hvalout.mat')
    x_val = mat['HT'] # array
    mat = sio.loadmat('data/DATA_Htestout.mat')
    x_test = mat['HT'] # array

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')
x_train = np.reshape(x_train, (len(x_train), img_channels, img_height, img_width))  # adapt this if using `channels_first` image data format
x_val = np.reshape(x_val, (len(x_val), img_channels, img_height, img_width))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), img_channels, img_height, img_width))  # adapt this if using `channels_first` image data format

In [4]:
file = 'CsiNet_'+(envir)+'_dim'+str(encoded_dim)
json_file = open('saved_model/model_%s.json'%file, 'r')
base_model_json = json_file.read()
json_file.close()
base_model = model_from_json(base_model_json)
# load weights into new model
base_model.load_weights("saved_model/model_%s.h5"%file)
print("Loaded base model from disk")
base_model.summary()

Loaded base model from disk
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2, 32, 32)]  0           []                               
                                                                                                  
 conv2d_1 (Conv2D)              (None, 2, 32, 32)    38          ['input_1[0][0]']                
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 2, 32, 32)   128         ['conv2d_1[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 leaky_re_lu_1 (LeakyReLU)      (None, 2, 32, 32)    0          

In [5]:
cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 32,
  'cluster_centroids_init': CentroidInitialization.KMEANS_PLUS_PLUS
}

# Helper function uses `cluster_weights` to make only 
# the Dense layers train with clustering
def apply_clustering_to_dense(layer):
  if isinstance(layer, tf.keras.layers.Dense):
    return cluster_weights(layer, **clustering_params)
  return layer

# Use `tf.keras.models.clone_model` to apply `apply_clustering_to_dense` 
# to the layers of the model.
clustered_model = tf.keras.models.clone_model(
    base_model,
    clone_function=apply_clustering_to_dense,
)

clustered_model.compile(optimizer='adam', loss='mse')

clustered_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2, 32, 32)]  0           []                               
                                                                                                  
 conv2d_1 (Conv2D)              (None, 2, 32, 32)    38          ['input_1[0][0]']                
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 2, 32, 32)   128         ['conv2d_1[1][0]']               
 rmalization)                                                                                     
                                                                                                  
 leaky_re_lu_1 (LeakyReLU)      (None, 2, 32, 32)    0           ['batch_normalization_1[1][

In [6]:
clustered_model.fit(x_train, x_train,
                epochs=8,
                batch_size=200,
                shuffle=True,
                validation_data=(x_val, x_val))

Epoch 1/8
500/500 [==============================] - 27s 41ms/step - loss: 5.3805e-04 - val_loss: 5.4318e-04
Epoch 2/8
500/500 [==============================] - 18s 35ms/step - loss: 5.3724e-04 - val_loss: 5.3727e-04
Epoch 3/8
500/500 [==============================] - 18s 36ms/step - loss: 5.3775e-04 - val_loss: 5.3756e-04
Epoch 4/8
500/500 [==============================] - 19s 37ms/step - loss: 5.3755e-04 - val_loss: 5.3829e-04
Epoch 5/8
500/500 [==============================] - 19s 38ms/step - loss: 5.3744e-04 - val_loss: 5.4352e-04
Epoch 6/8
500/500 [==============================] - 21s 42ms/step - loss: 5.3717e-04 - val_loss: 5.3641e-04
Epoch 7/8
500/500 [==============================] - 23s 45ms/step - loss: 5.3697e-04 - val_loss: 5.4215e-04
Epoch 8/8
500/500 [==============================] - 24s 48ms/step - loss: 5.3687e-04 - val_loss: 5.3769e-04


In [7]:
final_model = tfmot.clustering.keras.strip_clustering(clustered_model)
file = (envir)+'_dim'+str(encoded_dim)
# clustered_keras_file = ('omar_saved_model/clustered_model.h5')
# tf.keras.models.save_model(final_model, clustered_keras_file, include_optimizer=False)
# print('Saved clustered Keras model to:', clustered_keras_file)

In [8]:
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
clustered_tflite_model = converter.convert()

# Saving the model.
with open('optimized_models/%s/clustered_model_%s.tflite'%(file, file), 'wb') as f:
  f.write(clustered_tflite_model)


INFO:tensorflow:Assets written to: C:\Users\Omar\AppData\Local\Temp\tmpfo3icekz\assets


INFO:tensorflow:Assets written to: C:\Users\Omar\AppData\Local\Temp\tmpfo3icekz\assets


In [9]:
def get_gzipped_model_size(file):
  # It returns the size of the gzipped model in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [10]:
file = (envir)+'_dim'+str(encoded_dim)
clustered_size = (get_gzipped_model_size('optimized_models/%s/clustered_model_%s.tflite'%(file, file)))
print("clustered model in Mb:", clustered_size / float(2**20))

clustered model in Mb: 0.13439369201660156


In [11]:

# Passing the Keras model to the TF Lite Converter.
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
# Using float-16 quantization.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
# Converting the model.
clustered_fp16_quantized_tflite_model = converter.convert()

converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_clustered_tflite_model = converter.convert()

# Saving the model.
with open('optimized_models/%s/clustered_quant_model_%s.tflite'%(file, file), 'wb') as f:
  f.write(quantized_and_clustered_tflite_model)
with open('optimized_models/%s/clustered_fp16_quant_model_%s.tflite'%(file, file), 'wb') as f:
  f.write(clustered_fp16_quantized_tflite_model)


INFO:tensorflow:Assets written to: C:\Users\Omar\AppData\Local\Temp\tmpw8tst0e4\assets


INFO:tensorflow:Assets written to: C:\Users\Omar\AppData\Local\Temp\tmpw8tst0e4\assets


INFO:tensorflow:Assets written to: C:\Users\Omar\AppData\Local\Temp\tmpjd07d07e\assets


INFO:tensorflow:Assets written to: C:\Users\Omar\AppData\Local\Temp\tmpjd07d07e\assets


In [12]:
prune_quant_size = (get_gzipped_model_size('optimized_models/%s/clustered_quant_model_%s.tflite'%(file, file)))
prune_fp16_quant_size = (get_gzipped_model_size('optimized_models/%s/clustered_fp16_quant_model_%s.tflite'%(file, file)))
print("pruned quant model in Mb:", prune_quant_size / float(2**20))

pruned quant model in Mb: 0.10184097290039062
